# Analysis of dataset to assign label

The purpose of this subproject is ; in a simple and easy way; assign a better label for classification in kind of disaster and whether or not it is a joke.

As dataset is pre-labeled from human team; i will use this features like basis and guide for this process.

## Preparing enviroment to work

In [1]:
!pip install boto3
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
import re
from functools import reduce

import pandas as pd
import numpy as np
from boto3 import Session

In [4]:
disaster_db_df = pd.read_csv("/database.csv", encoding="latin9")
disaster_db_df.head(5)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one,choose_one:confidence,choose_one_gold,keyword,location,text,tweetid,userid
0,778243823,True,golden,156,NaN,Relevant,1.0000,Relevant,NaN,NaN,Just happened a terrible car crash,1.0,NaN
1,778243824,True,golden,152,NaN,Relevant,1.0000,Relevant,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,13.0,NaN
2,778243825,True,golden,137,NaN,Relevant,1.0000,Relevant,NaN,NaN,"Heard about #earthquake is different cities, s...",14.0,NaN
3,778243826,True,golden,136,NaN,Relevant,0.9603,Relevant,NaN,NaN,"there is a forest fire at spot pond, geese are...",15.0,NaN
4,778243827,True,golden,138,NaN,Relevant,1.0000,Relevant,NaN,NaN,Forest fire near La Ronge Sask. Canada,16.0,NaN


Based in data sample, i think the `choose_one`, `choose_one:confidence`, `choosen_one_gold`, `text` and `tweetid` are important for the analysis; i need to check if my hypothesis is true.

---



# Checking the mentioned features are relevant or not

### **`Choose_one` field**

In [5]:
disaster_db_df['choose_one'].value_counts()

Not Relevant    6187
Relevant        4673
Can't Decide      16
Name: choose_one, dtype: int64

In [6]:
disaster_db_df['choose_one'].isnull().value_counts()

False    10876
Name: choose_one, dtype: int64

That's ready to use ✅

### **`choose_one_gold` field**

In [7]:
disaster_db_df['choose_one_gold'].value_counts()

Relevant                      57
Not Relevant                  24
Relevant\nCan't Decide         4
Not Relevant\nCan't Decide     2
Name: choose_one_gold, dtype: int64

In [8]:
disaster_db_df['choose_one_gold'].isnull().value_counts()

True     10789
False       87
Name: choose_one_gold, dtype: int64

***QUESTION #1: Can i replace it with `choose_one` field?***

In [9]:
disaster_db_df.replace(np.nan, None)
disaster_db_df["choose_one_gold_v2"] = disaster_db_df["choose_one_gold"].apply(
    lambda x: x.split("\n")[-1] if type(x) is str else None
)
disaster_db_df["choose_one_gold_v2"].value_counts()

Relevant        57
Not Relevant    24
Can't Decide     6
Name: choose_one_gold_v2, dtype: int64

In [10]:
disaster_db_df["is_choose_one_gold_with_choose_one"] = disaster_db_df.apply( 
    lambda x: True if x["choose_one_gold_v2"] == x["choose_one_gold"] else False, 
    axis=1
  )
selected_choose_one_gold_df = disaster_db_df[ disaster_db_df["choose_one_gold_v2"].isin(["Relevant", "Not Relevant", "Can't Decide"])]
selected_choose_one_gold_df["is_choose_one_gold_with_choose_one"].value_counts()

True     81
False     6
Name: is_choose_one_gold_with_choose_one, dtype: int64

In [11]:
selected_choose_one_gold_df = selected_choose_one_gold_df[selected_choose_one_gold_df["is_choose_one_gold_with_choose_one"].isin([False])]
selected_choose_one_gold_df[["choose_one_gold_v2", "choose_one", "choose_one_gold", "is_choose_one_gold_with_choose_one"]]

,choose_one_gold_v2,choose_one,choose_one_gold,is_choose_one_gold_with_choose_one
9,Can't Decide,Relevant,Relevant\nCan't Decide,False
13,Can't Decide,Relevant,Relevant\nCan't Decide,False
15,Can't Decide,Relevant,Relevant\nCan't Decide,False
19,Can't Decide,Relevant,Relevant\nCan't Decide,False
42,Can't Decide,Not Relevant,Not Relevant\nCan't Decide,False
44,Can't Decide,Not Relevant,Not Relevant\nCan't Decide,False


In previous table, i see that all non-null value of `choose_one_gold` field are same with `choose_one` field.

Ok, i ***won't use this field*** because it have several null value and i have more value from `choose_one` ❌

### **`choose_one:confidence` field**

In [12]:
disaster_db_df['choose_one:confidence'].isnull().value_counts()

False    10876
Name: choose_one:confidence, dtype: int64

In [13]:
def assign_range_value(x):
  if 0.0 < x <= 0.5: return "0.0 - 0.5"
  elif 0.5 < x <= 0.75: return "0.5 - 0.75"
  elif 0.75 < x < 0.9: return "0.75 - 0.9"
  elif 0.9 < x <= 1.0: return "0.9 - 1.0"
  else: return "> 1.0"

disaster_db_df["range_confidence"] = disaster_db_df["choose_one:confidence"].apply(lambda x: assign_range_value(x))
disaster_db_df['range_confidence'].value_counts()

0.9 - 1.0     5066
0.75 - 0.9    3075
0.5 - 0.75    2597
0.0 - 0.5      138
Name: range_confidence, dtype: int64

i can see that 74.85% of tweets are trustworthy in with its value: That's good 😀. But the question is: **How is thise proportion is distribuite over `choosen_one` field?**

In [14]:
Relevant_group_df = disaster_db_df[disaster_db_df["choose_one"] == "Relevant"]
Relevant_group_df['range_confidence'].value_counts()

0.9 - 1.0     2273
0.75 - 0.9    1172
0.5 - 0.75    1148
0.0 - 0.5       80
Name: range_confidence, dtype: int64

In [15]:
Relevant_group_df = disaster_db_df[disaster_db_df["choose_one"] == "Not Relevant"]
Relevant_group_df['range_confidence'].value_counts()

0.9 - 1.0     2793
0.75 - 0.9    1903
0.5 - 0.75    1445
0.0 - 0.5       46
Name: range_confidence, dtype: int64

In [16]:
Relevant_group_df = disaster_db_df[disaster_db_df["choose_one"] == "Can't Decide"]
Relevant_group_df['range_confidence'].value_counts()

0.0 - 0.5     12
0.5 - 0.75     4
Name: range_confidence, dtype: int64

⚡ **Idea**: I can see the `Can't Decide` value in `choose_one` field is irrelevant for the amount, but i see that `0.5-0.75` range of other values of `choose_one` field can be renamed in `Can't Decide` value to get more amount: from 16 to 2609; that's good comparing other group amount 4.696(`Not Relevant`) and 3445(`Relevant`).

I won't use any value from `choose_one` field with `0.0-5.0` range because these values are irrelevant: they are not trustworthy.

### **`tweetid` and `text` fields**

In [17]:
disaster_db_df['tweetid'].isnull().value_counts()

False    10876
Name: tweetid, dtype: int64

In [18]:
disaster_db_df.astype({'tweetid': 'str'}).dtypes

_unit_id                                int64
_golden                                  bool
_unit_state                            object
_trusted_judgments                      int64
_last_judgment_at                      object
choose_one                             object
choose_one:confidence                 float64
choose_one_gold                        object
keyword                                object
location                               object
text                                   object
tweetid                                object
userid                                float64
choose_one_gold_v2                     object
is_choose_one_gold_with_choose_one       bool
range_confidence                       object
dtype: object

In [19]:
unique_tweet_id = set(disaster_db_df['tweetid'])
list_tweet_id = list(disaster_db_df['tweetid'])
print("unique value are %s from %s" % (len(unique_tweet_id), len(list_tweet_id)))

unique value are 1243 from 10876


In [20]:
disaster_db_df['tweetid'].value_counts()

6.290840e+17    197
6.290830e+17    190
6.290990e+17    166
6.290800e+17    157
6.290820e+17    152
               ... 
6.260600e+17      1
6.260750e+17      1
6.262370e+17      1
6.276670e+17      1
6.585000e+03      1
Name: tweetid, Length: 1243, dtype: int64

In [21]:
disaster_db_df["is_tweet_id_duplicated"] = disaster_db_df.duplicated(subset=['tweetid'])
duplicated_tweetit_db_df = disaster_db_df[disaster_db_df["is_tweet_id_duplicated"] == True]
duplicated_tweetit_db_df.sort_values(by = 'tweetid', inplace=True)
duplicated_tweetit_db_df[["tweetid", "text"]].head(20)

<ipython-input-21-2d003273006d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicated_tweetit_db_df.sort_values(by = 'tweetid', inplace=True)


,tweetid,text
10872,4.234000e+03,Police investigating after an e-bike collided ...
9066,6.260620e+17,Investigators say a fatal Virgin Galactic spac...
9076,6.260620e+17,#BBCLive Investigators say a fatal Virgin Gala...
9083,6.260630e+17,Investigators say a fatal Virgin Galactic spac...
9048,6.260650e+17,Investigators say a fatal Virgin Galactic spac...
1962,6.260760e+17,Schools in Western Uganda still Burning down B...
9047,6.260760e+17,Virgin galactic crash: early unlocking of brak...
9065,6.260760e+17,Virgin galactic crash: early unlocking of brak...
9049,6.260890e+17,Investigators rule catastrophic structural fai...
9089,6.260900e+17,NTSB: Virgin Galactic crash caused by structur...


i can see several points:


1.   Case #1 (based on 9083 and 9048 id): i can get difference values from `tweetids` field but same text apparently. 
2.   Case #2 : i can several duplicated value in `tweetids` field.
3.   Case #3 (based on 9076 and  9066) : i can get similar texts but with its differences from same value in `tweetids` field.

Based in these points i need to do following 🧰:


*   Remove URLs and arrobas.
*   Replace hashtag content with non-hashtag version.
*   Get the most long length version of text to cover more information.
*   Check the distribution of `choose one` field after that, to create a clean dataset (based in the ideas) and the label.


## Create a new and cleaning dataset

### Remove URLs and arrobas

In [62]:
def remove_urls(x):
  ''' Remove URLs from sentence'''
  word_list = x.split(" ")
  word_list = [ 
      word.lower() for word in word_list \
      if re.search("^http(s)*:\/\/", word) is None
    ]
  return " ".join(word_list)


def remove_arrobas(x):
  ''' Remove arrobas from sentence'''
  word_list = x.split(" ")
  clean_words = []
  for word in word_list:
    if re.search("^@", word) is None:
      clean_words.append(word.lower())
    else:
      clean_words.append("<user>")
  return " ".join(clean_words)


def abs_remove_urls_and_arroba(*functions):
    return reduce(lambda f, g: lambda x: g(f(x)), functions, lambda x: x)

remove_urls_and_arroba = abs_remove_urls_and_arroba(remove_arrobas, remove_urls)


disaster_db_df["text_without_url_and_arroba"] = disaster_db_df["text"].apply(lambda x: remove_urls_and_arroba(x))
clean_step_1_tweetit_db_df = disaster_db_df[disaster_db_df["is_tweet_id_duplicated"] == True]
clean_step_1_tweetit_db_df.sort_values(by = 'tweetid', inplace=True)
clean_step_1_tweetit_db_df[["text_without_url_and_arroba", "text"]].head(20)

<ipython-input-62-696b1a8a3de6>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_step_1_tweetit_db_df.sort_values(by = 'tweetid', inplace=True)


,text_without_url_and_arroba,text
10872,police investigating after an e-bike collided ...,Police investigating after an e-bike collided ...
9076,#bbclive investigators say a fatal virgin gala...,#BBCLive Investigators say a fatal Virgin Gala...
9066,investigators say a fatal virgin galactic spac...,Investigators say a fatal Virgin Galactic spac...
9083,investigators say a fatal virgin galactic spac...,Investigators say a fatal Virgin Galactic spac...
9048,investigators say a fatal virgin galactic spac...,Investigators say a fatal Virgin Galactic spac...
9047,virgin galactic crash: early unlocking of brak...,Virgin galactic crash: early unlocking of brak...
1962,schools in western uganda still burning down b...,Schools in Western Uganda still Burning down B...
9065,virgin galactic crash: early unlocking of brak...,Virgin galactic crash: early unlocking of brak...
9049,investigators rule catastrophic structural fai...,Investigators rule catastrophic structural fai...
9089,ntsb: virgin galactic crash caused by structur...,NTSB: Virgin Galactic crash caused by structur...


### Replace hashtag value with non-hashtag value

In [63]:
def replace_with_non_hashtag_value(x):
  ''' Replace hashtag content with non-hashtag version'''
  word_list = x.split(" ")
  clean_words = []
  for word in word_list:
    if re.search("^#", word) is None:
      clean_words.append(word.lower())
    else:
      clean_words.append(word[1:].lower())
  return " ".join(clean_words)


disaster_db_df["text_with_non_hashtag"] = disaster_db_df["text_without_url_and_arroba"].apply(lambda x: replace_with_non_hashtag_value(x))
clean_step_2_tweetit_db_df = disaster_db_df[disaster_db_df["is_tweet_id_duplicated"] == True]
clean_step_2_tweetit_db_df.sort_values(by = 'tweetid', inplace=True)
clean_step_2_tweetit_db_df[["text_with_non_hashtag", "text"]].head(20)

<ipython-input-63-48f05293f318>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_step_2_tweetit_db_df.sort_values(by = 'tweetid', inplace=True)


,text_with_non_hashtag,text
10872,police investigating after an e-bike collided ...,Police investigating after an e-bike collided ...
9076,bbclive investigators say a fatal virgin galac...,#BBCLive Investigators say a fatal Virgin Gala...
9066,investigators say a fatal virgin galactic spac...,Investigators say a fatal Virgin Galactic spac...
9083,investigators say a fatal virgin galactic spac...,Investigators say a fatal Virgin Galactic spac...
9048,investigators say a fatal virgin galactic spac...,Investigators say a fatal Virgin Galactic spac...
9047,virgin galactic crash: early unlocking of brak...,Virgin galactic crash: early unlocking of brak...
1962,schools in western uganda still burning down b...,Schools in Western Uganda still Burning down B...
9065,virgin galactic crash: early unlocking of brak...,Virgin galactic crash: early unlocking of brak...
9049,investigators rule catastrophic structural fai...,Investigators rule catastrophic structural fai...
9089,ntsb: virgin galactic crash caused by structur...,NTSB: Virgin Galactic crash caused by structur...


In [64]:
def remove_with_non_hashtag_value(x):
  ''' Remove hashtag value'''
  word_list = x.split(" ")
  clean_words = []
  for word in word_list:
    if re.search("^#", word) is None:
      clean_words.append(word.lower())

  return " ".join(clean_words)

disaster_db_df["text_without_hashtag_version"] = disaster_db_df["text_without_url_and_arroba"].apply(lambda x: remove_with_non_hashtag_value(x))
clean_step_2_1_tweetit_db_df = disaster_db_df[disaster_db_df["is_tweet_id_duplicated"] == True]
clean_step_2_1_tweetit_db_df.sort_values(by = 'tweetid', inplace=True)
clean_step_2_1_tweetit_db_df[["text_without_hashtag_version", "text"]].head(20)

<ipython-input-64-59fe4d89c0ce>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_step_2_1_tweetit_db_df.sort_values(by = 'tweetid', inplace=True)


,text_without_hashtag_version,text
10872,police investigating after an e-bike collided ...,Police investigating after an e-bike collided ...
9076,investigators say a fatal virgin galactic spac...,#BBCLive Investigators say a fatal Virgin Gala...
9066,investigators say a fatal virgin galactic spac...,Investigators say a fatal Virgin Galactic spac...
9083,investigators say a fatal virgin galactic spac...,Investigators say a fatal Virgin Galactic spac...
9048,investigators say a fatal virgin galactic spac...,Investigators say a fatal Virgin Galactic spac...
9047,virgin galactic crash: early unlocking of brak...,Virgin galactic crash: early unlocking of brak...
1962,schools in western uganda still burning down b...,Schools in Western Uganda still Burning down B...
9065,virgin galactic crash: early unlocking of brak...,Virgin galactic crash: early unlocking of brak...
9049,investigators rule catastrophic structural fai...,Investigators rule catastrophic structural fai...
9089,ntsb: virgin galactic crash caused by structur...,NTSB: Virgin Galactic crash caused by structur...


### Get the most long length version of text to cover more information.

In [65]:
def get_lenght_text(x):
  ''' Get the lenght of words in "text" field'''
  return len(x.split(" "))

disaster_db_df["amount_words_in_cleaning_text"] = disaster_db_df["text_without_hashtag_version"].apply( lambda x : get_lenght_text(x))
disaster_db_df.sort_values(by = 'amount_words_in_cleaning_text', inplace=True)
cleaning_disaster_db_df = disaster_db_df[['choose_one', 'choose_one:confidence', 'text','tweetid', 'range_confidence', 
  'is_tweet_id_duplicated', 'amount_words_in_cleaning_text', "text_without_hashtag_version"]]
cleaning_disaster_db_df.rename(columns={
    "text_without_hashtag_version": "cleaning_text",
    "choose_one:confidence": "confidence",
    "tweetid": "tweet_id"
  }, inplace=True)
cleaning_disaster_db_df.head(15)

<ipython-input-65-fdce3a786b39>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaning_disaster_db_df.rename(columns={


,choose_one,confidence,text,tweet_id,range_confidence,is_tweet_id_duplicated,amount_words_in_cleaning_text,cleaning_text
9487,Relevant,0.8018,Truth...\nhttps://t.co/n1K5nlib9X\n#News\n#BBC...,6.290990e+17,0.75 - 0.9,True,1,truth...\nhttps://t.co/n1k5nlib9x\n#news\n#bbc...
9490,Relevant,0.4048,Truth...\nhttps://t.co/4ZQrsAQrRT\n#News\n#BBC...,6.291010e+17,0.0 - 0.5,True,1,truth...\nhttps://t.co/4zqrsaqrrt\n#news\n#bbc...
9486,Not Relevant,1.0000,Truth...\nhttps://t.co/k44tL3rfMy\n#News\n#BBC...,6.290990e+17,0.9 - 1.0,True,1,truth...\nhttps://t.co/k44tl3rfmy\n#news\n#bbc...
1191,Not Relevant,0.5960,Stats http://t.co/U7vavyrGv9,6.291060e+17,0.5 - 0.75,True,1,stats
9454,Not Relevant,0.5988,Truth...\nhttps://t.co/beJfTYgJIL\n#News\n#BBC...,6.290990e+17,0.5 - 0.75,True,1,truth...\nhttps://t.co/bejftygjil\n#news\n#bbc...
190,Not Relevant,0.5031,Aftershock https://t.co/jV8ppKhJY7,6.289000e+17,0.5 - 0.75,False,1,aftershock
45,Not Relevant,1.0000,Awesome!,5.000000e+01,0.9 - 1.0,False,1,awesome!
5851,Relevant,1.0000,Yesterday's #hailstorm! #boston #cambridge htt...,6.290260e+17,0.9 - 1.0,True,1,yesterday's
7295,Not Relevant,1.0000,Err:509,6.289470e+17,0.9 - 1.0,True,1,err:509
7400,Not Relevant,0.7995,Obliterated,6.290890e+17,0.75 - 0.9,True,1,obliterated


In [66]:
cleaning_disaster_db_df.sort_values(
    by=["cleaning_text", "confidence", "amount_words_in_cleaning_text"],
    ascending = False, 
    inplace=True
  )
cleaning_disaster_db_df.head(10)


<ipython-input-66-aaca77229732>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaning_disaster_db_df.sort_values(


,choose_one,confidence,text,tweet_id,range_confidence,is_tweet_id_duplicated,amount_words_in_cleaning_text,cleaning_text
1394,Not Relevant,1.0000,å€} New Ladies Shoulder Tote #Handbag Faux Lea...,6.290670e+17,0.9 - 1.0,True,14,å€} new ladies shoulder tote faux leather hobo...
9663,Not Relevant,0.8006,#TornadoGiveaway åÊ #thebookclub åÊ Join in!! ...,6.291100e+17,0.75 - 0.9,True,4,åê åê join in!!
9694,Relevant,0.6006,#TornadoGiveaway åÊ #thebookclub åÊ Join in!! ...,6.291100e+17,0.5 - 0.75,True,4,åê åê join in!!
2351,Not Relevant,0.6018,åÈMGN-AFRICAåš pin:263789F4 åÈ Correction: Ten...,6.293480e+17,0.5 - 0.75,True,13,åèmgn-africaåš pin:263789f4 åè correction: ten...
4006,Relevant,1.0000,å¬'Only the sea knows how many are dead' @MSF_...,6.290910e+17,0.9 - 1.0,True,18,å¬'only the sea knows how many are dead' <user...
1414,Not Relevant,1.0000,å_? New Ladies Shoulder Tote #Handbag Faux Lea...,6.290490e+17,0.9 - 1.0,True,12,å_? new ladies shoulder tote faux leather hobo...
5410,Relevant,1.0000,#NJTurnpike å_ #NJTurnpike Reopens Hours After...,6.290920e+17,0.9 - 1.0,True,7,å_ reopens hours after truck fire in?
1416,Not Relevant,1.0000,ÛŒ? New Ladies Shoulder Tote #Handbag Faux Le...,6.290330e+17,0.9 - 1.0,True,12,ûœ? new ladies shoulder tote faux leather hob...
5420,Relevant,1.0000,Û÷We Can HelpÛª Says Denver Firefighter Work...,6.290920e+17,0.9 - 1.0,True,12,û÷we can helpûª says denver firefighter work...
8990,Relevant,0.6027,Û÷SexistÛª peer review causes storm online h...,6.293600e+17,0.5 - 0.75,True,8,û÷sexistûª peer review causes storm online v...


In [67]:
print(cleaning_disaster_db_df.shape)

(10876, 8)


In [69]:
cleaning_disaster_db_df["is_duplicated_cleaning_text"] = cleaning_disaster_db_df.duplicated(subset=['cleaning_text'])
cleaning_disaster_db_df["is_duplicated_cleaning_text"].value_counts()

<ipython-input-69-9edba0a8448f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaning_disaster_db_df["is_duplicated_cleaning_text"] = cleaning_disaster_db_df.duplicated(subset=['cleaning_text'])


False    9668
True     1208
Name: is_duplicated_cleaning_text, dtype: int64

In [71]:
### WARNING: i will reduce the db to get only validated records
cleaning_disaster_db_df = cleaning_disaster_db_df[cleaning_disaster_db_df["is_duplicated_cleaning_text"] == False]
print(cleaning_disaster_db_df.shape)

(9668, 9)


Great!: after the cleaning process, the DB preserves the 88.89% of original records 🥳🎉

## Verify the new distribution of value and validate our ideas

In [76]:
cleaning_disaster_db_df['range_confidence'].value_counts()

0.9 - 1.0     4498
0.75 - 0.9    2743
0.5 - 0.75    2306
0.0 - 0.5      121
Name: range_confidence, dtype: int64

After the cleaning process, the data preserve the 66.58% of the original data like trustworthy value: that's not bad 👏. I think in first view the previous idea can be applied.

In [77]:
Relevant_group_df = cleaning_disaster_db_df[cleaning_disaster_db_df["choose_one"] == "Relevant"]
Relevant_group_df['range_confidence'].value_counts()

0.9 - 1.0     1834
0.5 - 0.75    1009
0.75 - 0.9     995
0.0 - 0.5       70
Name: range_confidence, dtype: int64

In [78]:
Relevant_group_df = cleaning_disaster_db_df[cleaning_disaster_db_df["choose_one"] == "Not Relevant"]
Relevant_group_df['range_confidence'].value_counts()

0.9 - 1.0     2664
0.75 - 0.9    1748
0.5 - 0.75    1293
0.0 - 0.5       41
Name: range_confidence, dtype: int64

In [79]:
Relevant_group_df = cleaning_disaster_db_df[cleaning_disaster_db_df["choose_one"] == "Can't Decide"]
Relevant_group_df['range_confidence'].value_counts()

0.0 - 0.5     10
0.5 - 0.75     4
Name: range_confidence, dtype: int64

after the cleaning process; i can apply the idea to get `Can't decide` with 2413, `Not Relevant` with 4.412 and `Relevant` with 2.843.